In [2]:
from src.gpt2 import GPT2

In [3]:
a = GPT2(16, 2, 8, 28*28, 1,1)

In [4]:
a

GPT2(
  (token_embeddings): Embedding(1, 16)
  (position_embeddings): Embedding(784, 16)
  (layers): ModuleList(
    (0): Block(
      (ln_1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): Linear(in_features=16, out_features=16, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=16, out_features=16, bias=True)
        (1): GELU()
        (2): Linear(in_features=16, out_features=16, bias=True)
      )
    )
    (1): Block(
      (ln_1): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (ln_2): LayerNorm((16,), eps=1e-05, elementwise_affine=True)
      (attn): MultiheadAttention(
        (out_proj): Linear(in_features=16, out_features=16, bias=True)
      )
      (mlp): Sequential(
        (0): Linear(in_features=16, out_features=16, bias=True)
        (1): GELU()
        (2): Linear(in_features=16, out_features=16, bias=True)
 